#  Sokoban Satisfeito
## Avaliação Contínua 4
### Introdução à Inteligência Artificial, edição 2024/25, Dep. Informática FCUL
### Entrega: 16 de Dezembro (1m antes da meia-noite)

***

## Introdução e Objetivos

Nesta avaliação voltamos ao nosso amigo Sokoban. Vamos ajudá-lo a descobrir estados em que não é possível arrumar todas as caixas em objetivos, e assim evitar que ele ande à procura de uma solução que não existe. Faremos isto através de modelização e resolução de Problemas de Satisfação de Restrições (CSPs - *Constraint Satisfaction Problems*).

Vão precisar dos módulos seguintes (distribuídos juntamente com o enunciado):
* `search.py`
* `searchPlus.py`
* `utils.py`
* `sokoban_aval4.py`
* `csp_v3.py`


#### O puzzle Sokoban

Recordamos que o puzzle Sokoban é jogado numa grelha discreta de células quadradas, onde cada célula é chão navegável ou uma parede. Algumas das células de chão contêm caixas e outras estão marcadas como lugares de armazenamento das caixas. O jogador (Sokoban) está confinado ao tabuleiro e pode movimentar-se ortogonalmente para células adjacentes, que sejam navegáveis e estejam vazias. O Sokoban pode empurrar caixas que estejam ao seu lado, para células vazias adjacentes. As caixas não podem ser empurradas se ao seu lado, na direção do movimento, estiver uma parede ou outra caixa. O puzzle fica resolvido quando todos os lugares de armazenamento forem ocupados por caixas.

#### Cantos, becos e outros *deadlocks*

Algumas células vazias, apesar de navegáveis, são consideradas proibidas porque, não sendo lugares de armazenamento, caso o Sokoban empurre uma caixa para lá causa um *deadlock* porque já não conseguirá voltar a movê-la e assim nunca conseguirá terminar o puzzle. Na Avaliação Contínua 2 o código incluía a deteção de casas que causam este tipo de *deadlock* com a função `its_a_trap` que deteta cantos e becos. Nesta avaliação vamos detetar, com recurso a CSPs, outros tipos de *deadlocks*.

## Função `csp_possivel_solucao`
 
Em primeiro lugar, vamos usar uma função fornecida por nós, chamada `possivel_solucao`, que recebe como input

1. As caixas (coordenadas das suas localizações);
2. Uma tabela formada pelos pares (célula,objectivos) em que cada entrada da tabela (dicionário) indica quais os objectivos alcançáveis (as suas coordenadas) pela caixa se estivesse na célula.

e devolve uma possível atribuição de caixas a objetivos, ou None caso seja impossível arrumar todas as caixas nos objectivos. Esta função não garante detectar todos os casos de solução impossível, mas todos os casos que detecta como impossíveis serão mesmo impossíveis. A função será integrada no método `actions` para que, quando é detetado o *deadlock*, não seja devolvida qualquer ação possível.

Vejamos um exemplo ilustrativo:

<img src="exemplo_puzzle.png" alt="Drawing" style="width: 600px;"/>

Na imagem da esquerda, os goals (locais de armazenamento) estão identificados com números de 1 a 4. As caixas são as casas com quadrados verdes e o Sokoban é o círculo verde. Na imagem da direita vê-se a atribuição, a cada célula navegável, de uma lista de goals que podem ser atingidos a partir dessa célula. [ * ] significa a lista com todos os goals, [1,2,3,4]. 

A função `possivel_solucao` será como especificada abaixo, e vai usar a **vossa** função **`csp_possivel_solucao`** da seguinte forma:

In [ ]:
def possivel_solucao(caixas,goals_alcancaveis):
    csp_sokoban1 = csp_possivel_solucao(caixas,goals_alcancaveis) # <--- a vossa função csp_possivel_solucao 
    r = backtracking_search(csp_sokoban1, inference = forward_checking)
    return r

Para implementarem a vossa função **`csp_possivel_solucao`**, devem formular o problema como um CSP. A função recebe como input exatamente o mesmo que a função `possivel_solucao` e deve retornar como output uma instância de um problema CSP. Notem que o problema pode ser visto como um problema de coloração de mapas em que:

- As células são as variáveis (as regiões num mapa);
- Os domínios são dados pela lista de goals alcançáveis pelas respectivas células (as cores para colorir o mapa);
- Duas células são vizinhas se partilharem algum dos goals (se forem regiões contíguas);
- Duas células vizinhas não podem ter o mesmo valor (duas regiões contíguas não podem ter a mesma cor).

Lembrem-se que ambos os inputs são dados, i.e., as caixas e a atribuição de células a listas de goals (os goals alcançáveis).

Iremos testar se as vossas variáveis, domínios, vizinhos e restrições estão corretas, da forma exemplificada em baixo. Recordamos que

-  \#  é uma parede
-  @  é o Sokoban
-  $  é uma caixa
-  o  é um lugar de armazenamento
-  .  é uma casa vazia

e que as coordenadas das casas (usadas em `alcancaveis`, por exemplo) começam em (0,0) no canto superior esquerdo. Para recordarem mais coisas podem recorrer às Avaliações Contínuas 1 e/ou 2.

In [ ]:
from sokoban_aval4 import *

linha1= "    ####\n"
linha2= "  ##...#\n"
linha3= "###....#\n"
linha4= "#o..$#@#\n"
linha5= "#oo$.$.#\n"
linha6= "###o.$.#\n"
linha7= "  ###..#\n"
linha8= "    ####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8

alcancaveis={(1, 4): [], (1, 5): [], (1, 6): [], (2, 3): [], (2, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 6): [], (3, 1): [(3, 1)], (3, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 6): [], (4, 1): [(4, 1)], (4, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 6): [], (5, 3): [(5, 3)], (5, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 6): [], (6, 5): [], (6, 6): []}
                   
s = Sokoban(situacaoInicial=mundoS)
caixas = s.initial['caixas']
csp_sokoban1 = csp_possivel_solucao(caixas,alcancaveis)

print('Variáveis:',sorted(csp_sokoban1.variables))
print('Domínios:',dict(sorted(csp_sokoban1.domains.items())))
sorted_neighbors = {key: sorted(values) for key, values in sorted(csp_sokoban1.neighbors.items())}
print('Vizinhos:',dict(sorted(sorted_neighbors.items())))
print('Restrição obedecida?',csp_sokoban1.constraints((3,4),(3,1),(4,3),(3,1)))
print('Restrição obedecida?',csp_sokoban1.constraints((3,4),(3,1),(4,3),(4,1)))

Claro que não vai funcionar se ainda não programaram a função `csp_possivel_solucao`. O resultado da célula anterior deve ser:

    Variáveis: [(3, 4), (4, 3), (4, 5), (5, 5)]
    Domínios: {(3, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 5): [(3, 1), (4, 1), (4, 2), (5, 3)]}
    Vizinhos: {(3, 4): [(4, 3), (4, 5), (5, 5)], (4, 3): [(3, 4), (4, 5), (5, 5)], (4, 5): [(3, 4), (4, 3), (5, 5)], (5, 5): [(3, 4), (4, 3), (4, 5)]}
    Restrição obedecida? False
    Restrição obedecida? True

Também iremos testar o resultado da `possivel_solucao` quando usa a vossa função, desta forma:

In [ ]:
from sokoban_aval4 import *

linha1= "    ####\n"
linha2= "  ##...#\n"
linha3= "###....#\n"
linha4= "#o..$#@#\n"
linha5= "#oo$.$.#\n"
linha6= "###o.$.#\n"
linha7= "  ###..#\n"
linha8= "    ####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8

alcancaveis={(1, 4): [], (1, 5): [], (1, 6): [], (2, 3): [], (2, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 6): [], (3, 1): [(3, 1)], (3, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 6): [], (4, 1): [(4, 1)], (4, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 6): [], (5, 3): [(5, 3)], (5, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 6): [], (6, 5): [], (6, 6): []}
                   
s = Sokoban(situacaoInicial=mundoS)
caixas = s.initial['caixas']
result = possivel_solucao(caixas,alcancaveis) # <--- usa a vossa função csp_possivel_solucao
if result != None:
    result = dict(sorted(result.items()))
print(result)


Aqui o resultado devia ser:

    {(3, 4): (4, 2), (4, 3): (5, 3), (4, 5): (3, 1), (5, 5): (4, 1)}

Vejam um caso em que não há solução possível:

In [ ]:
from sokoban_aval4 import *

linha1= "    ####\n"
linha2= "  ##...#\n"
linha3= "###....#\n"
linha4= "#o..$#@#\n"
linha5= "#oo$.$.#\n"
linha6= "###o...#\n"
linha7= "  ###$.#\n"
linha8= "    ####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8

alcancaveis={(1, 4): [], (1, 5): [], (1, 6): [], (2, 3): [], (2, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (2, 6): [], (3, 1): [(3, 1)], (3, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (3, 6): [], (4, 1): [(4, 1)], (4, 2): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 3): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (4, 6): [], (5, 3): [(5, 3)], (5, 4): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 5): [(3, 1), (4, 1), (4, 2), (5, 3)], (5, 6): [], (6, 5): [], (6, 6): []}
                   
s = Sokoban(situacaoInicial=mundoS)
caixas = s.initial['caixas']
result = possivel_solucao(caixas,alcancaveis) # <--- usa a vossa função csp_possivel_solucao
if result != None:
    result = dict(sorted(result.items()))
print(result)


O resultado da célula anterior deve ser:

    None

## Função `csp_find_alcancaveis_1goal`

No exemplo anterior, a variável `alcancaveis` era dada, mas queremos ser nós a obtê-la automaticamente com base no puzzle. Vamos formular este problema também como um CSP. Na verdade, vamos ter um CSP por cada goal, implementado numa função chamada `find_alcancaveis_1goal` que recebe como input

1. Uma instância do problema Sokoban;
2. As coordenadas de um goal desse problema.

e devolve, para cada célula navegável, a informação sobre se uma caixa nessa célula poderia ser empurrada até ao goal dado (1 se sim, 0 se não).

A função `find_alcancaveis_1goal` será como especificada abaixo, e vai usar a **vossa** função **`csp_find_alcancaveis_1goal`** da seguinte forma:

In [ ]:
def find_alcancaveis_1goal(s,goal):
    csp_sokoban2 = csp_find_alcancaveis_1goal(s,goal) # <--- a vossa função csp_find_alcancaveis_1goal 
    r = backtracking_search(csp_sokoban2, order_domain_values = number_ascending_order, inference = forward_checking)    
    return {} if r == None else r

Para implementarem a função **`csp_find_alcancaveis_1goal`** devem formular o problema como um CSP em que:

- As células são as variáveis;
- Os domínios são a possibilidade (1) ou não (0) de uma caixa partir da célula e chegar ao goal;
- Duas células são vizinhas se forem adjacentes (na vertical ou horizontal) e se for possível empurrar uma caixa de uma para a outra, num dos sentidos ou nos dois. Se forem adjacentes, mas nenhuma caixa pode ser empurrada de uma para a outra, então não são vizinhas.
- As restrições, tantos as unárias como as binárias, ficam por vossa conta!

Reparem que este CSP pode devolver soluções que, apesar de válidas no contexto do problema, não ajudam o Sokoban tanto como outras. Por exemplo, se duas soluções válidas tiverem como única diferença o valor atribuído a uma das células (0 num caso, 1 no outro), a solução que mais ajuda o Sokoban é aquela que tem o 0, pois permite-lhe mais facilmente detetar um *deadlock*. É por isso que a função `find_alcancaveis_1goal` faz sempre a procura com `order_domain_values = number_ascending_order`, como especificado acima.

Iremos testar a vossa função quando é usada pela `find_alcancaveis_1goal`, desta forma:

In [ ]:
from sokoban_aval4 import *

linha1= "#####\n"
linha2= "#...#\n"
linha3= "#.@.#\n"
linha4= "#.$.#\n"
linha5= "#.o.#\n"
linha6= "#####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6

s = Sokoban(situacaoInicial=mundoS)
result = find_alcancaveis_1goal(s,(4,2)) # <--- usa a vossa função csp_find_alcancaveis_1goal
result = dict(sorted(result.items()))
print(result)

Mais uma vez, só funciona quando completarem a função `csp_find_alcancaveis_1goal`. O resultado da célula anterior deve ser:

{(1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 2): 1, (2, 3): 0, (3, 1): 0, (3, 2): 1, (3, 3): 0, (4, 1): 0, (4, 2): 1, (4, 3): 0}

## Função `find_alcancaveis_all_goals`

No exemplo anterior o puzzle só tinha um goal, mas normalmente tem mais do que um. Precisamos agora de uma função **vossa** chamada **`find_alcancaveis_all_goals`** que chama a `find_alcancaveis_1goal` tantas vezes quantas o número de goals do puzzle. Recebe uma instância do problema Sokoban e devolve a variável `alcancaveis` que é construída a partir dos resultados das várias chamadas.

Mostramos em baixo a estrutura que aconselhamos para esta função. Ordenem os goals como mostrado, para que os resultados sejam exatamente como esperado.

In [ ]:
def find_alcancaveis_all_goals(s):
    sorted_goals = sorted(list(s.goal))
    pass # o vosso código aqui
    return result_alcancaveis

Iremos testar a vossa função da seguinte forma:

In [ ]:
from sokoban_aval4 import *

linha1= "#####\n"
linha2= "#...#\n"
linha3= "#o@.#\n"
linha4= "#.$$#\n"
linha5= "#.o.#\n"
linha6= "#####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6

s = Sokoban(situacaoInicial=mundoS)
result = find_alcancaveis_all_goals(s)
result = dict(sorted(result.items()))
print(result)

A célula anterior devia dar:

    {(1, 1): [], (1, 2): [], (1, 3): [], (2, 1): [(2, 1)], (2, 2): [(2, 1), (4, 2)], (2, 3): [], (3, 1): [(2, 1)], (3, 2): [(2, 1), (4, 2)], (3, 3): [], (4, 1): [], (4, 2): [(4, 2)], (4, 3): []}

## Tudo junto no Sokoban Satisfeito

Implementadas as três funções **`csp_possivel_solucao`**, **`csp_find_alcancaveis_1goal`** e **`find_alcancaveis_all_goals`**, podemos finalmente juntar tudo para ajudar o Sokoban a realizar procuras mais eficientes. Usamos o algoritmo A* com a heurística `h_util` da Avaliação Contínua 2, que se encontra no código incluído e que não devem alterar. Vejam como, num exemplo tão simples de um problema que tem solução, mesmo assim o Sokoban Satisfeito consegue reduzir o número de estados visitados, comparado com o Sokoban que não tem ajuda para detetar *deadlocks*.

In [ ]:
from sokoban_aval4 import *

linha1= "#####\n"
linha2= "#.oo#\n"
linha3= "#.@.#\n"
linha4= "#.$$#\n"
linha5= "#...#\n"
linha6= "#####\n"
mundoS=linha1+linha2+linha3+linha4+linha5+linha6


# Sokoban Satisfeito:

s = Sokoban(situacaoInicial=mundoS,plug_in_function=possivel_solucao)
s.alcancaveis = find_alcancaveis_all_goals(s)
res, exp = astar_search_plus_count(s,s.h_util)
if res == None:
    print('No solution!')
else:
    print('Explorados com CSP:',exp)
    print(res.solution())

    
# Sokoban sem ajuda: 

def sempre_possivel(a,b):
    '''Função que, ao contrário da csp_possivel_solucao, diz sempre que há solução e por isso não ajuda nada.'''
    return True

s = Sokoban(situacaoInicial=mundoS,plug_in_function=sempre_possivel)
s.alcancaveis = {} # não sabemos nada sobre os goals alcançáveis
res, exp = astar_search_plus_count(s,s.h_util)
if res == None:
    print('No solution!')
else:
    print('Explorados sem CSP:',exp)
    print(res.solution())

O resultado da célula anterior deve ser:

    Explorados com CSP: 22
    ['W', 'S', 'S', 'E', 'N', 'N', 'S', 'S', 'E', 'N', 'N']
    Explorados sem CSP: 24
    ['W', 'S', 'S', 'E', 'N', 'N', 'S', 'S', 'E', 'N', 'N']

## Submissão

### Quiz
Cada grupo deve completar as implementações pedidas e testá-las no link do *quiz* **Avaliação Contínua 4** que está na página da disciplina, introduzindo aí o vosso código. 

Esse *quiz* é constituído por uma única pergunta, onde as implementações das diferentes funções são avaliadas através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo no total 2.15 valores. Alguns testes são específicos para a função `csp_possivel_solucao` (valendo 0.6 valores), outros para a função `csp_find_alcancaveis_1goal` (valendo 1 valor). Outros testes avaliam o par de funções `csp_find_alcancaveis_1goal` e `find_alcancaveis_all_goals` a trabalhar juntas (valendo 0.25 valores), e finalmente os últimos testes avaliam todas as funções a trabalhar em conjunto na procura A* (valendo 0.3 valores). Os testes visíveis representam 30% da nota e os invisíveis 70%.

Podem ir verificando o código (botão check) e submetendo as vezes que quiserem (por ambos os elementos do grupo), sendo a submissão com melhor nota a que será considerada. Qualquer tentativa não manualmente submetida é automaticamente submetida no fecho do prazo.

### Prazo
A submissão fecha às 23:59 de 2a feira, 16 de Dezembro

### Ficheiro Python
Simultaneamente é necessario submeter o ficheiro Python que contém todas as vossas funções, na página da disciplina. Só queremos uma submissão por grupo. Esse ficheiro deve chamar-se *SokobanSatisfeito_IIA_24_25_grupoXX.py* em que substituem XX pelo identificador do grupo. 